In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
p = Path("../data/")
files = list(p.glob("*.csv"))
sorted(files)

In [ ]:
for file in files:
    df = pd.read_csv(file)
    print(file, ":", df.shape)

# Sequences


Columns:

- target_id: unique identifier
- sequence
- temporal_cutoff
- description
- all_sequences


In [ ]:
from rnafold.report import report

report()

In [ ]:
from rnafold.config import Settings

sequences = pd.read_csv(Settings.sequences.test)
sequences

In [ ]:
# Get one sample
ROW = 0

sample = sequences.iloc[ROW].to_dict()
sample

# Submission


In [ ]:
submission = pd.read_csv(p / "sample_submission.csv")
submission

In [ ]:
labels_val = pd.read_csv(Settings.labels.val)
labels_val

In [ ]:
labels_train = pd.read_csv(Settings.labels.train)
labels_train

In [ ]:
labels_train[labels_train.isna().sum(axis=1) > 0]

- there are NaN values in the train dataset -> to investigate

In [ ]:
import pandas as pd
import numpy as np

df = labels_train

# Define N
N = 5

# Identify rows with NaN
nan_indices = df.index[df.isna().any(axis=1)]

# Get the range of indices to keep
keep_indices = set()
for idx in nan_indices:
    for offset in range(-N, N + 1):  # Include N rows before and after
        new_idx = idx + offset
        if 0 <= new_idx < len(df):  # Ensure within bounds
            keep_indices.add(new_idx)

# Extract the sub-dataframe
sub_df = df.loc[sorted(keep_indices)].reset_index(drop=True)

print(sub_df)
sub_df.to_csv("weird.csv")